# 전국 신규 민간 아파트 분양가격 동향
* 주택분양보증을 받아 분양한 전체 민간 신규아파트 분양가격 동향
* https://www.data.go.kr/dataset/3035522/fileData.do

In [1]:
import pandas as pd

In [2]:
pre_sale = pd.read_csv('data/지역별_규모별_㎡당_평균_분양가격_천원__18.4월.csv', encoding='euc-kr')
pre_sale.shape

(2635, 5)

In [3]:
pre_sale.head()

,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721
4,서울,전용면적 102㎡초과,2015,10,5879


In [10]:
# 분양가격에 결측치가 많이 있다.
pre_sale.isnull().sum()

지역명          0
규모구분         0
연도           0
월            0
분양가격(㎡)    115
dtype: int64

In [15]:
pre_sale.describe()

,연도,월
count,2635.000000,2635.000000
mean,2016.548387,6.419355
std,0.836383,3.670751
min,2015.000000,1.000000
25%,2016.000000,3.000000
50%,2017.000000,6.000000
75%,2017.000000,10.000000
max,2018.000000,12.000000


In [16]:
# 2017년 데이터만 본다.
pre_sale_2017 = pre_sale.loc[pre_sale['연도'] == 2017]
pre_sale_2017.shape

(1020, 5)